THIS IS WHERE I TEST STUFF

In [1]:
import pandas as pd
import numpy as np
import cv2
from src.dataset import CropsDataset, ObjectDetectorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from src.utils import collate_fn
from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay

le = LabelEncoder()
le.fit(['human-ride-bicycle', 'human-walk-bicycle', 'human-hold-bicycle', 'human-ride-motorcycle', 'human-walk-motorcycle'])
dataset = CropsDataset(anno_file=r'../annotations_hoi_frame.csv', img_dir = r'../Image Dataset', label_encoder=le)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

# Iterate through dataloader
#image, target, coord = next(iter(dataloader))
#plt.imshow(image[0].numpy().transpose(1, 2 , 0))
#print(le.inverse_transform(target[0].numpy()))


In [2]:
# Extract features and labels
data_list = []
data_list2 = []
for batch in dataloader:
    crop, label, coord = batch
    data_list.append((coord,label))
    data_list2.append((crop,label))
    

##### Training clf on bbox coordinates

In [ ]:
print(data_list[533][1])
print(len(data_list))
print(le.inverse_transform(np.ravel(data_list[0][1])))

# Separate features and labels in separate lists
# Reshaping labels cause classifier expect dim <= 2
features, labels = zip(*data_list)
labels = np.array(labels).reshape(-1)
features = np.array([coord[0] for coord in features])
print(labels.shape)
#print(features)
print(features.shape)
#features = [image[0].reshape(-1).numpy() for image in features]

# Split the data into training and testing sets with random seed
seed = 1
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=seed)

# Creating a RandomForestClassifier with 100 trees with random seed
# Trying to fit the features (cropped frames of human+object)
rf_classifier = RandomForestClassifier(n_estimators=1000, random_state=seed)
rf_classifier.fit(features,labels)

# Make predictions on the test set
predictions = rf_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

"""
### Observation ###
Due to different shapes of the cropped frames
i cant train the classifier on the features.

I might try padding the crops?

Trying to pass coordinates instead, works.
"""

##### Training clf on cropped imgs

In [19]:
print(data_list2[533][1])
print(len(data_list2))
print(le.inverse_transform(np.ravel(data_list2[0][1])))

# Separate features and labels in separate lists
# Reshaping labels cause classifier expect dim <= 2
features, labels = zip(*data_list2)
labels = np.array(labels).reshape(-1)
#features = np.array([coord[0] for coord in features])
#print(labels.shape)
#print(features)
#print(features.shape)
features = [image[0].numpy().reshape(-1) for image in features]
#features = [image[0].numpy() for image in features]
print(features[0].shape)


#print(features[0][0].shape)
#for img in features:
#    print(img[0].shape)

# Find the maximum height and width among all crops
max_height = max(img.shape[1] for img in features)
max_width = max(img.shape[2] for img in features)

print(f"(ch,{max_height},{max_width})")
test = np.empty((3, max_height, max_width))
print("test",test.shape)

# Pad each crop to match the maximum height and width
features = [np.pad(img,pad_width=1) for img in features]

# Split the data into training and testing sets with random seed
seed = 1
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=seed)

# Creating a RandomForestClassifier with 100 trees with random seed
# Trying to fit the features (cropped frames of human+object)
rf_classifier = RandomForestClassifier(n_estimators=1000, random_state=seed)
rf_classifier.fit(features,labels)

# Make predictions on the test set
predictions = rf_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


(tensor([1], dtype=torch.int32),)
535
['human-ride-motorcycle']
(210,)


IndexError: tuple index out of range